In [2]:
import pandas as pd
import os
from IPython.display import display, Markdown # For explicitly controlling display and adding text
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.patches import Patch
import seaborn as sns
import numpy as np
from scipy import stats
import statsmodels.formula.api as smf
from datetime import timedelta
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
file_path_1 = r'C:\Users\REALD\Documents\RIA\caiso_curtailment.csv'
file_path_2 = r'C:\Users\REALD\Documents\RIA\caiso_standard.csv'
file_path_3 = r'C:\Users\REALD\Documents\RIA\caiso_renewable_forecast.csv'
file_path_4 = r'C:\Users\REALD\Documents\RIA\caiso_load_forecast.csv'
file_path_5 = r'C:\Users\REALD\Documents\RIA\caiso_lmp.csv'

curtailment = pd.read_csv(file_path_1)
data = pd.read_csv(file_path_2)
renewable = pd.read_csv(file_path_3)
load = pd.read_csv(file_path_4)
lmp = pd.read_csv(file_path_5)

In [4]:
for x in [curtailment, data, renewable, load, lmp]:
    x.drop('interval_start_local', axis=1, inplace=True)
    x.drop('interval_end_local', axis=1, inplace=True)
    x.drop('interval_end_utc', axis=1, inplace=True)
    x['interval_start_utc'] = pd.to_datetime(x['interval_start_utc'])

In [5]:
renewable['publish_time_utc'] = pd.to_datetime(renewable['publish_time_utc'])

In [6]:
renewable = renewable.loc[renewable.groupby('interval_start_utc')['publish_time_utc'].idxmax()]

In [7]:
lmp_avg = lmp.groupby('interval_start_utc')[['lmp', 'energy', 'congestion', 'loss', 'ghg']].mean().reset_index()

In [8]:
# list of the columns you want diffs for
cols = ['lmp', 'energy', 'congestion', 'loss', 'ghg']

# compute max – min per interval
lmp_diffs = (
    lmp
    .groupby('interval_start_utc')[cols]
    .max()
    .subtract(
        lmp.groupby('interval_start_utc')[cols].min()
    )
    .rename(columns=lambda c: f"{c}_diff")
    .reset_index()
)

# diffs now has columns: interval_start_utc, lmp_diff, energy_diff, congestion_diff, loss_diff, ghg_diff
lmp_diffs.head(5)

,interval_start_utc,lmp_diff,energy_diff,congestion_diff,loss_diff,ghg_diff
0,2022-01-01 08:00:00+00:00,1.877925,0.0,0.0,1.877925,0.0
1,2022-01-01 09:00:00+00:00,0.000000,0.0,0.0,0.000000,0.0
2,2022-01-01 10:00:00+00:00,1.535882,0.0,0.0,1.535883,0.0
3,2022-01-01 11:00:00+00:00,2.526922,0.0,0.0,2.526930,0.0
4,2022-01-01 12:00:00+00:00,2.743608,0.0,0.0,2.743603,0.0


In [9]:
curtailmentx = curtailment.groupby(['interval_start_utc', 'fuel_type'])[['curtailment_mwh', 'curtailment_mw']].sum().reset_index()

In [10]:
curtailment_grouped = curtailmentx.groupby(['interval_start_utc'])[['curtailment_mwh', 'curtailment_mw']].sum().reset_index()

In [11]:
curtailment_solar = curtailmentx[curtailmentx['fuel_type'] == 'Solar'].copy()

In [12]:
curtailment_wind = curtailmentx[curtailmentx['fuel_type'] == 'Wind'].copy()

In [13]:
load['net_load_forecast'] = load['load_forecast'] - renewable['solar_mw'] - renewable['wind_mw']

In [14]:
load.drop(['publish_time_local', 'publish_time_utc', 'tac_area_name'], axis=1, inplace=True)
load.columns.tolist()

['interval_start_utc', 'load_forecast', 'net_load_forecast']

In [15]:
renewable.drop(['location', 'publish_time_local', 'publish_time_utc'], axis=1, inplace=True)
renewable.columns.tolist()

['interval_start_utc', 'solar_mw', 'wind_mw']

In [16]:
curtailment_grouped.drop(['curtailment_mwh'], axis=1, inplace=True)
curtailment_grouped.rename(columns={'curtailment_mw': 'total_curtailment_mw'}, inplace=True)
curtailment_grouped.columns.tolist()

['interval_start_utc', 'total_curtailment_mw']

In [17]:
curtailment_solar.rename(columns={'curtailment_mw' : 'solar_curtailment_mw'}, inplace=True)
curtailment_solar.drop(['fuel_type', 'curtailment_mwh'], axis=1, inplace=True)
curtailment_solar.columns.tolist()

['interval_start_utc', 'solar_curtailment_mw']

In [18]:
curtailment_wind.rename(columns={'curtailment_mw' : 'wind_curtailment_mw'}, inplace=True)
curtailment_wind.drop(['fuel_type', 'curtailment_mwh'], axis=1, inplace=True)
curtailment_wind.columns.tolist()

['interval_start_utc', 'wind_curtailment_mw']

In [19]:
data['total_generation'] = data.filter(like='fuel_mix.').sum(axis=1)

In [20]:
df = pd.merge(data, load, on='interval_start_utc', how='inner')
df = pd.merge(df, renewable, on='interval_start_utc', how='inner')
df = pd.merge(df, curtailment_grouped, on='interval_start_utc', how='left')
df = pd.merge(df, curtailment_solar, on='interval_start_utc', how='left')
df = pd.merge(df, curtailment_wind, on='interval_start_utc', how='left')
df = pd.merge(df, lmp_avg, on='interval_start_utc', how='inner')
df = pd.merge(df, lmp_diffs, on='interval_start_utc', how='inner')

In [21]:
# 1b) any interval with no row in curtailment_grouped now has NaN → make that 0
df['total_curtailment_mw'] = df['total_curtailment_mw'].fillna(0)
df['solar_curtailment_mw'] = df['solar_curtailment_mw'].fillna(0)
df['wind_curtailment_mw'] = df['wind_curtailment_mw'].fillna(0)

# 1c) compute percent
df['percent'] = df['total_curtailment_mw'] / df['total_generation']

In [22]:
df['interval_pacific'] = df['interval_start_utc'].dt.tz_convert('America/Los_Angeles')

In [23]:
df.drop(['interval_start_utc'], axis=1, inplace=True)

In [24]:
# 2a) Extract day‐of‐week as a string
df['day_of_week'] = df['interval_pacific'].dt.day_name()  # Monday, Tuesday, …

# 2b) Extract month as integer (1–12) and hour
df['month'] = df['interval_pacific'].dt.month          # 1, 2, …, 12
df['hour']  = df['interval_pacific'].dt.hour           # 0, 1, …, 23

# 3) One‐hot encode
dow_dummies    = pd.get_dummies(df['day_of_week'], prefix='dow')
month_dummies  = pd.get_dummies(df['month'],          prefix='month')
hour_dummies   = pd.get_dummies(df['hour'],           prefix='hour')

# 4) Append back onto your DataFrame
df = pd.concat([df, dow_dummies, month_dummies, hour_dummies], axis=1)

In [25]:
df.head(5)

,load_forecast.load_forecast,net_load,renewables,renewables_to_load_ratio,load.load,fuel_mix.solar,fuel_mix.wind,fuel_mix.geothermal,fuel_mix.biomass,fuel_mix.biogas,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,22664.759167,18229.750000,6543.916667,0.295315,22159.083333,-31.916667,3961.250000,879.000000,274.583333,210.000000,...,False,False,False,False,False,False,False,False,False,False
1,21823.094167,17812.750000,6184.000000,0.289143,21387.333333,-32.000000,3606.583333,877.666667,280.750000,210.833333,...,False,False,False,False,False,False,False,False,False,False
2,21163.079167,17657.333333,5710.583333,0.274644,20792.666667,-32.000000,3167.333333,877.333333,282.166667,210.500000,...,False,False,False,False,False,False,False,False,False,False
3,20790.173125,17527.750000,5237.166667,0.256553,20413.583333,-33.166667,2919.000000,877.500000,279.666667,210.750000,...,False,False,False,False,False,False,False,False,False,False
4,20750.285833,17761.583333,5028.333333,0.246314,20414.333333,-33.083333,2685.833333,879.250000,282.333333,211.000000,...,False,False,False,False,False,False,False,False,False,False


In [56]:
import pandas as pd
import mysql.connector
from mysql.connector import errorcode

conn_cfg = {
    "user":     "app_user",
    "password": "",              # blank, as you set it
    "host":     "127.0.0.1",
    "database": "database210",
    "use_pure": True
}

DTYPE_MAP = {
    'int32':   'INT',
    'float64': 'DOUBLE',
    'bool':    'TINYINT(1)',
    'object':  'VARCHAR(255)',
    'datetime64[ns]': 'DATETIME'
}
def infer_col_type(series: pd.Series) -> str:
    return DTYPE_MAP.get(str(series.dtype), 'VARCHAR(255)')

def write_df_to_mysql(df: pd.DataFrame, table_name: str):
    # build CREATE TABLE DDL
    cols_ddl = []
    for col in df.columns:
        col_type = infer_col_type(df[col])
        null_flag = 'NULL' if df[col].isnull().any() else 'NOT NULL'
        cols_ddl.append(f"`{col}` {col_type} {null_flag}")
    create_ddl = f"""
    CREATE TABLE IF NOT EXISTS `{table_name}` (
      {',\n  '.join(cols_ddl)}
    ) ENGINE=InnoDB;
    """

    cnx = mysql.connector.connect(**conn_cfg)
    cur = cnx.cursor()

    try:
        cur.execute(create_ddl)
    except mysql.connector.Error as err:
        if err.errno != errorcode.ER_TABLE_EXISTS_ERROR:
            cur.close(); cnx.close()
            raise

    cols       = ", ".join(f"`{c}`" for c in df.columns)
    placeholders = ", ".join(["%s"] * len(df.columns))
    insert_sql   = f"INSERT INTO `{table_name}` ({cols}) VALUES ({placeholders})"

    data = []
    for row in df.itertuples(index=False):
        vals = []
        for v in row:
            if isinstance(v, pd.Timestamp):
                v = v.to_pydatetime()
            vals.append(int(v) if isinstance(v, bool) else v)
        data.append(tuple(vals))

    cur.executemany(insert_sql, data)
    cnx.commit()
    print(f"Inserted {cur.rowcount} rows into `{table_name}`.")

    cur.close()
    cnx.close()

write_df_to_mysql(df, table_name='curtailment')

Inserted 25535 rows into `curtailment`.


In [58]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
df.dtypes

load_forecast.load_forecast                                float64
net_load                                                   float64
renewables                                                 float64
renewables_to_load_ratio                                   float64
load.load                                                  float64
fuel_mix.solar                                             float64
fuel_mix.wind                                              float64
fuel_mix.geothermal                                        float64
fuel_mix.biomass                                           float64
fuel_mix.biogas                                            float64
fuel_mix.small_hydro                                       float64
fuel_mix.coal                                              float64
fuel_mix.nuclear                                           float64
fuel_mix.natural_gas                                       float64
fuel_mix.large_hydro                                       flo